In [305]:
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.misc import derivative
import pandas as pd
import matplotlib.pyplot as plt
import julia

from calibration_helpers import load_data

In [131]:
# Load simulation data
inputpath = '../sensitivity/sensitivity_runs/input_data/'
outputpath = 'calibrationdata/'
n_threads = 16
run_nr = 9

df_input, df_output = load_data(inputpath, outputpath, n_threads, return_as_np=False, run_nr=run_nr)

In [132]:
df_input.head()

,sim_nr,κ_upper,ω,ϵ,α_cp,p_f,prog
0,1,0.016731,0.607195,0.004777,0.420845,0.612024,0.958734
1,2,0.025614,0.787970,0.018947,0.920375,0.262146,0.069943
2,3,0.043017,0.156076,0.053897,0.407351,0.410693,-0.014409
3,4,0.005877,0.613369,0.025382,0.971976,0.942169,0.392512
4,5,0.000625,0.177624,0.093463,0.531359,0.281305,-0.942203


In [133]:
df_output.head()

,sim_nr,GDP_100,GDP_101,GDP_102,GDP_103,GDP_104,GDP_105,GDP_106,GDP_107,GDP_108,...,em_451,em_452,em_453,em_454,em_455,em_456,em_457,em_458,em_459,em_460
0,1.0,-0.002278,-0.712838,-0.126260,1.137066,0.514139,0.238982,-1.214088,-0.485300,0.946937,...,130.563809,130.034282,137.813585,130.630564,130.127925,129.974645,130.371658,138.821657,130.593431,132.986633
1,2.0,-0.504616,-0.821751,-0.785637,0.000987,1.013080,-0.420483,1.017981,0.136271,1.040422,...,734.083977,737.944245,739.610067,740.829398,743.148928,744.165481,745.398268,748.997820,755.221652,759.759419
2,3.0,-0.098923,-0.073446,-0.237771,0.221862,0.337703,-0.275593,-0.032419,-0.249349,-0.110371,...,180.708069,166.720712,172.383156,157.672057,183.245038,169.482975,167.727131,169.459350,172.164705,167.821169
3,4.0,0.813734,-1.589643,3.178776,-1.797173,2.209950,0.735034,3.130155,2.701971,-0.251060,...,107.273697,107.103721,107.292378,107.352468,107.389431,107.481518,107.478413,107.609686,107.621334,107.640903
4,5.0,-0.806535,-1.309255,0.934620,0.608261,-0.198540,0.041941,-1.349040,0.573423,0.074261,...,96.775153,104.076315,96.706367,100.247513,99.010026,100.019726,97.963592,101.960777,96.387543,100.933807


In [134]:
def gen_sim_moments():

    moments = {}

    GDP_dat = df_output[[col for col in df_output if col.startswith('GDP')]]

    moments['gdp_1'] = np.mean(GDP_dat, axis=1)
    moments['gdp_2'] = np.std(GDP_dat, axis=1)
    moments['gdp_3'] = stats.skew(GDP_dat, axis=1)
    moments['gdp_4'] = stats.kurtosis(GDP_dat, axis=1)

    U_dat = df_output[[col for col in df_output if col.startswith('U')]]

    moments['u_1'] = np.mean(U_dat, axis=1).to_numpy()

    dU_dat = U_dat.replace(0, 0.001).pct_change(axis=1).fillna(0).to_numpy()[:, 1:]

    moments['du_1'] = np.mean(dU_dat, axis=1)
    moments['du_2'] = np.std(dU_dat, axis=1)
    moments['du_3'] = stats.skew(dU_dat, axis=1)
    moments['du_4'] = stats.kurtosis(dU_dat, axis=1)

    # em_dat = df_output[[col for col in df_output if col.startswith('em')]]

    moments['em2030'] = df_output['em_220']
    moments['em2040'] = df_output['em_340']
    moments['em2050'] = df_output['em_460']

    df_sim_moments = pd.DataFrame(moments)
    df_sim_moments.head()

    return df_sim_moments

In [135]:
df_sim_moments = gen_sim_moments()
df_sim_moments.head()

,gdp_1,gdp_2,gdp_3,gdp_4,u_1,du_1,du_2,du_3,du_4,em2030,em2040,em2050
0,-0.025358,0.720609,-0.066463,0.720162,0.924114,0.000014,0.002672,0.084414,-0.393746,110.323372,119.920776,132.986633
1,0.140111,1.003387,0.077004,0.354969,0.022586,0.000209,0.057003,0.599420,3.248122,202.022384,453.538695,759.759419
2,-0.016188,0.817432,-0.186018,0.312197,0.939489,0.000152,0.007356,0.059155,0.643597,107.588360,141.285410,167.821169
3,0.400643,1.793415,0.184501,1.993859,0.000014,0.009423,0.225218,17.761260,330.429872,97.991956,100.521367,107.640903
4,0.005197,0.808929,0.014918,-0.095829,0.813649,0.000218,0.019859,0.266777,0.219136,99.477816,103.818156,100.933807


In [312]:
def gen_true_moments():

    # TODO: wrap this in a function
    true_dat = {
        'GDP_1st':[ 0.482],
        'GDP_2nd': [0.47],
        'GDP_3rd': [-2.272],
        'GDP_4th': [11.945],
        'U_1st': [0.06097],
        'dU_1st': [-0.003],
        'dU_2nd': [0.003],
        'dU_3rd': [1.066],
        'dU_4th': [2.138],
        'em2030': [112.72],
        'em2040': [121.818],
        'em2050': [127.27]
    }
    df_true_moments = pd.DataFrame(true_dat)
    df_true_moments.to_csv('true_moments.csv', index=False)
    return df_true_moments

In [313]:
df_true_moments = gen_true_moments()
df_true_moments.head()

,GDP_1st,GDP_2nd,GDP_3rd,GDP_4th,U_1st,dU_1st,dU_2nd,dU_3rd,dU_4th,em2030,em2040,em2050
0,0.482,0.47,-2.272,11.945,0.06097,-0.003,0.003,1.066,2.138,112.72,121.818,127.27


In [291]:
X_labels = {
    "α_cp": [0.6, 1.0],
    "prog": [-1.0, 1.0],
    "ω": [0.0, 1.0],
    "λ": [0.0, 1.0],
    "κ_upper": [0.0, 0.05],
    "p_f": [0.0, 1.0]
}

In [281]:
rescale = np.array([100., 1., 0.1, 0.1, 100., 1., 1., 1., 1., 1., 1., 1.])
rescale = np.array([100., 10., 0., 1., 100., 0., 0., 0., 10., 1., 1., 1.])

In [306]:
class MSE:
    def __init__(self, df_true_moments, df_sim_params, df_sim_moments, X_labels, rescale):
        self.df_true_moments = df_true_moments
        self.df_sim_params = df_sim_params 
        self.df_sim_moments = df_sim_moments
        self.rescale = rescale
        self.W = np.eye(df_true_moments.shape[1])
        self.X_labels = X_labels
        self.jl = julia.Julia(compiled_modules=False)

        self.jl.include("bayesian_calibration.jl")

    def compute_W(self):
        sim_moments = df_sim_moments.to_numpy()
        S = np.cov(sim_moments.T)
        self.W = np.linalg.inv(S)

    def minimize_MSE(self):

        bounds = np.array(list(self.X_labels.values()))
        print(bounds)

        optimal_theta = minimize(
                            self.run_model,
                            x0=np.array([0.8, 0., 0.8, 0.7, 0.005, 0.2]),
                            method='Nelder-Mead',
                            bounds=bounds,
                            tol=1e-8,
                            options={'maxiter': 10, 'disp': True}
                        )
        print(optimal_theta)

        # min_theta = []
        # min_row = []
        # min_err = []
        # min_crit = np.inf

        # for idx, row in self.df_sim_moments.iterrows():
        #     err, crit = self.compute_criterion(row.to_numpy() * self.rescale)

        #     if crit < min_crit:
        #         min_crit = crit
        #         min_theta = self.df_sim_params.iloc[idx].to_numpy()
        #         min_row = row
        #         min_err = err

        # # print(min_theta)
        # for i,col in enumerate(self.df_sim_params):
        #     print(f"{col}: {min_theta[i]}")
        # print(min_crit)
        # print(min_row)
        # print(min_err ** 2)

    def run_model(self, theta):

        # Run model
        gdp, u, em = self.jl.run_model(theta, list(self.X_labels.keys()))

        print(gdp, u, em)

        # Compute criterion
        return self.compute_criterion(m_sim)

    def compute_criterion(self, m_sim):
        """
        Computes the errors between the empirical moment vector and the 
        simulation vectors.
        """
        m_true = self.df_true_moments.to_numpy()[0] * self.rescale
        error = m_sim - m_true
        return error, error @ self.W @ error.T

In [307]:
mse_optimizer = MSE(df_true_moments, df_input, df_sim_moments, X_labels, rescale)
mse_optimizer.minimize_MSE()

CalledProcessError: Command '['julia', '-e', '...']' returned non-zero exit status 1.